# Part 2: Power Method

In [1]:
import numpy as np

#### 1. Implement the power method with normalization.

In [139]:
def power_method(matrix: np.ndarray, maximum_iterations: int):
    x_k = np.random.rand(matrix.shape[1])
    lmbda = 0

    for _ in range(maximum_iterations):
        z_k1 = np.dot(matrix, x_k)
        x_k = z_k1 / np.linalg.norm(z_k1)
        i = np.argmax(z_k1)
        lmbda = z_k1[i]/x_k[i]

    return lmbda, x_k
A = np.array([[5, -1, -2, 0], [-1, 3, -2, -1], [-2, -2, 5, 0], [0, -1, 0, 4]])
print(f"Largest eigenvalue: {power_method(A, 100)[0]}, associated eigenvector: {power_method(A, 100)[1]}")

Largest eigenvalue: 7.179335040371524, associated eigenvector: [ 0.59084255  0.24322828 -0.76543608 -0.07650288]


#### 2. Extend your function, to a new one, such that it takes an extra argument $\epsilon = 10^{−5}$. Keep track on the number of iterations that you need to reach the precision $|λ(k+1) − λ(k)| ≤ ε$.


In [ ]:
# def power_method(matrix: np.ndarray, maximum_iterations: int, acceptable_error: float):
#     x_k = np.random.rand(matrix.shape[1])
#     lmbda = 0

#     for _ in range(maximum_iterations):
#         z_k1 = np.dot(matrix, x_k)
#         x_k = z_k1 / np.linalg.norm(z_k1)
#         i = np.argmax(z_k1)
#         lmbda = z_k1[i]/x_k[i]

#     return lmbda, x_k